# **Práctica Flujos de Datos**
- Detección de Concept Drift.
- Clasificación.
- Agrupamiento.
- Tratamiento de Texto.
- Ensembles.

In [9]:
import pandas as pd
import numpy as np
import river
from river import datasets
from river import tree
from river import metrics
#from river import preprocessing
from river import evaluate
from river import drift
import time

## **Base de Datos**
https://www.kaggle.com/datasets/mariumfaheem666/spam-sms-classification-using-nlp

Se ha elegido un conjunto de datos utilizado para la detección de SPAM, en este la primera columna indica si el SMS recibido es SPAM o no y la siguiente contiene el texto del SMS. A partir de la segunda columna del texto, se han obtenido los siguientes atributos: el número de caracteres del SMS, el número de palabras, número de caracteres alfanuméricos, número de caracteres no alfanuméricos, número de símbolos de divisas, número de mayúsculas, de exclamaciones, de interrogaciones y de urls en cada mensaje. Estas características se han elegido ya que a nuestro parecer son importantes a la hora de decidir si un SMS es SPAM o no.

In [10]:
# Carga de Datos
df = pd.read_csv('spam_SMS_ampliado.csv')
df.head(5)
df.columns

Index(['spam', 'text', 'num_caracteres', 'num_palabras', 'num_alfabeticos',
       'num_numericos', 'num_no_alfanum', 'num_divisas', 'num_mayusculas',
       'num_exclamaciones', 'num_interrogaciones', 'num_urls'],
      dtype='object')

## Creación de un Modelo

In [20]:
# crear, entrenar y evaluar modelo
model_standard = tree.HoeffdingTreeClassifier()
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()
model_extreme = tree.ExtremelyFastDecisionTreeClassifier()
#metric = metrics.Accuracy()
metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            #'text': row['text'],
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)  # realiza una predicción
    model_standard.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
     
print('Standard', metric)

Adaptive F1: F1: 86.65%


In [22]:
# crear, entrenar y evaluar modelo
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()
#metric = metrics.Accuracy()
metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)  # realiza una predicción
    model_adaptive.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
     
print('Adaptive', metric)

Adaptive F1: 87.09%


In [28]:
# crear, entrenar y evaluar modelo
model_extreme = tree.ExtremelyFastDecisionTreeClassifier()
#metric = metrics.Accuracy()
metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)  # realiza una predicción
    model_extreme.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
     
print('Adaptive', metric)

AttributeError: 'NoneType' object has no attribute 'merit'

In [11]:
try:
    for index, row in df.iterrows():
        # Extrae características (X) y etiqueta (y)
        X = {
            'text': row['text'],
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

        y = row['spam'] 
        
        print(f"Datos recibidos: {X}, Etiqueta: {y}")
        
        # Aquí se aplicarían los algoritmos online
    
        time.sleep(0.5)  # Se espera medio segundo para simular el flujo de datos
        
except KeyboardInterrupt:
    print('Proceso detenido.')

Datos recibidos: {'text': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'num_caracteres': 111, 'num_palabras': 20, 'num_alfabeticos': 83, 'num_numericos': 0, 'num_no_alfanum': 28, 'num_divisas': 0, 'num_mayusculas': 3, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': 'Ok lar... Joking wif u oni...', 'num_caracteres': 29, 'num_palabras': 6, 'num_alfabeticos': 18, 'num_numericos': 0, 'num_no_alfanum': 11, 'num_divisas': 0, 'num_mayusculas': 2, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'num_caracteres': 155, 'num_palabras': 28, 'num_alfabeticos': 97, 'num_numericos': 25, 'num_no_alfanum': 33, 'num_divisas': 0, 'num_mayusculas': 10, 'num_exclamaciones